In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
!rm -rf joeynmt

In [26]:
# installing necessary applications for 
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!sudo add-apt-repository "deb http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/"
!sudo apt-get update
!sudo apt-get -y install cuda

# Install JoeyNMT
! git clone https://github.com/may-/joeynmt.git
! cd joeynmt; pip3 install .
# Install Pytorch with GPU support v1.9.0
! pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

--2021-09-12 13:50:27--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu1804.pin’

cuda-ubuntu1804.pin 100%[===================>]     190  --.-KB/s    in 0s      

2021-09-12 13:50:28 (9.47 MB/s) - ‘cuda-ubuntu1804.pin’ saved [190/190]

Executing: /tmp/apt-key-gpghome.oRcINv1QNQ/gpg.1.sh --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
gpg: requesting key from 'https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub'
gpg: key F60F4B3D7FA2AF80: "cudatools <cudatools@nvidia.com>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Er

In [29]:
import os
from os import path
tag = "pmi_nhs_da_retrain"
tokenized_data = "/content/drive/MyDrive/DA/updated_data_tok_files"
src_lang = "en"
tgt_lang = "hi"
bpe_operations = 5000
bpe_file = "len_full.bpe.codes." + str(bpe_operations)
bpe_folder = src_lang + "_" + tgt_lang + "_" + tag + "_" + str(bpe_operations)
bpe_data = "/content/drive/MyDrive/DA/" + bpe_folder + "/"

os.environ["src_lang"] = src_lang
os.environ["tgt_lang"] = tgt_lang
os.environ["tokenized_data"] = tokenized_data
os.environ["bpe_file"] = bpe_file
os.environ["bpe_operations"] = str(bpe_operations)
os.environ["jnmt_data_path"] = path.join("joeynmt", "data", src_lang + tgt_lang)

baseline_model_data = "/content/drive/MyDrive/DA/en_hi_pmi_nhs_baseline_5000/model_data"

os.environ["baseline_model_data"] = baseline_model_data

check_pnt = "/content/drive/MyDrive/DA/model_pmi_nhs_baseline_100epochs/enhi_transformer_pmi_nhs_baseline/best.ckpt"

! mkdir -p $jnmt_data_path
! mkdir -p $bpe_data

In [ ]:
!mkdir health_domain_data
!cp $tokenized_data/health_concat_filtered_tok.* $tokenized_data/test_tok* /content/health_domain_data

In [ ]:
!wc -l /content/health_domain_data/*

   6582 /content/health_domain_data/health_concat_filtered_tok.en
   6582 /content/health_domain_data/health_concat_filtered_tok.hi
    598 /content/health_domain_data/test_tok.en
    598 /content/health_domain_data/test_tok.hi
  14360 total


In [ ]:
import pandas as pd

def create_df(src_file, tgt_file):
  src = []
  tgt = []

  with open(src_file, 'r') as src_, open(tgt_file, 'r') as tgt_:
    src_lines = src_.readlines()
    tgt_lines = tgt_.readlines()

  for line_index in range(len(src_lines)):
    src.append(src_lines[line_index].strip('\n'))
    tgt.append(tgt_lines[line_index].strip('\n'))

  df = pd.DataFrame({'src_sentence': src,
                     'tgt_sentence': tgt})

  return df

In [ ]:
df_train_health = create_df("/content/health_domain_data/health_concat_filtered_tok.en", "/content/health_domain_data/health_concat_filtered_tok.hi" )

In [ ]:
df_train_health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6582 entries, 0 to 6581
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   src_sentence  6582 non-null   object
 1   tgt_sentence  6582 non-null   object
dtypes: object(2)
memory usage: 103.0+ KB


In [ ]:
### splitting data 500 rows from train data for validation purpose
from sklearn.model_selection import train_test_split

df_train, df_dev = train_test_split(df_train_health, test_size=0.076, random_state=42, shuffle=True)

df_dev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501 entries, 611 to 5706
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   src_sentence  501 non-null    object
 1   tgt_sentence  501 non-null    object
dtypes: object(2)
memory usage: 11.7+ KB


In [ ]:
def create_tok_files(input_df, dataset_name):
  src_tok_file = dataset_name + "_tok.en"
  tgt_tok_file = dataset_name + "_tok.hi"
  src_content = input_df['src_sentence'].to_list()
  tgt_content = input_df['tgt_sentence'].to_list()

  with open(src_tok_file, 'w') as src, open(tgt_tok_file, 'w') as tgt:
    for src_line, tgt_line in zip(src_content, tgt_content):
      src.write(src_line + "\n")
      tgt.write(tgt_line + "\n")


In [ ]:
create_tok_files(df_train, "train")
create_tok_files(df_dev, "dev")

In [ ]:
!cp /content/health_domain_data/*test* .

In [27]:
### Learn BPEs on training data
from os import path

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$src_lang < train_tok.$src_lang > train.bpe.$src_lang
! subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$tgt_lang < train_tok.$tgt_lang > train.bpe.$tgt_lang

! subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$src_lang < dev_tok.$src_lang > dev.bpe.$src_lang
! subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$tgt_lang < dev_tok.$tgt_lang > dev.bpe.$tgt_lang


! subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$src_lang < test_tok.$src_lang > test.bpe.$src_lang
! subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$tgt_lang < test_tok.$tgt_lang > test.bpe.$tgt_lang


In [30]:
!cp *.bpe.* $jnmt_data_path

!cp *.bpe.* $bpe_data

!cp $baseline_model_data/vocab.txt $jnmt_data_path

In [31]:
model_type = "bpe"
use_cuda = True

# Create the Joey NMT config file

config = """
name: "{name}_transformer"

data:
    src: "{src_lang}"
    trg: "{tgt_lang}"
    train: "data/{name}/train.{model_type}"
    dev:   "data/{name}/dev.{model_type}"
    test:   "data/{name}/test.{model_type}"
    level: "{model_type}"
    lowercase: False
    max_sent_length: 60
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "{check_pnt}"
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    learning_rate_warmup: 1000 
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 512
    batch_type: "token"
    eval_batch_size: 300
    eval_batch_type: "token"
    eval_metric: "bleu"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 100            # TODO: Set to at least once per epoch.
    logging_freq: 100
    model_dir: "models/{name}_transformer_{tag}"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: {use_cuda}
    max_output_length: 130
    print_valid_sents: [0, 1, 2, 3]
    keep_best_ckpts: 3
    
model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name = src_lang+tgt_lang, src_lang = src_lang, tgt_lang = tgt_lang, model_type = model_type, use_cuda = use_cuda, tag = tag, check_pnt = check_pnt)
with open("joeynmt/configs/transformer_{name}.yaml".format(name = src_lang+tgt_lang), 'w') as f:
    f.write(config)

In [32]:
! pip install --upgrade sacrebleu==2.0.0

In [33]:
# Train the model
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src_lang$tgt_lang.yaml

2021-09-12 13:53:42,548 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-09-12 13:53:42,601 - INFO - joeynmt.data - Loading training data...
2021-09-12 13:53:42,713 - INFO - joeynmt.data - Building vocabulary...
2021-09-12 13:53:43,137 - INFO - joeynmt.data - Loading dev data...
2021-09-12 13:53:43,143 - INFO - joeynmt.data - Loading test data...
2021-09-12 13:53:43,149 - INFO - joeynmt.data - Data loaded.
2021-09-12 13:53:43,149 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-09-12 13:53:43,346 - INFO - joeynmt.model - Enc-dec model built.
2021-09-12 13:53:45,232 - INFO - joeynmt.training - Total params: 12422400
2021-09-12 13:53:50,628 - INFO - joeynmt.training - Loading model from /content/drive/MyDrive/DA/model_pmi_nhs_baseline_100epochs/enhi_transformer_pmi_nhs_baseline/best.ckpt
2021-09-12 13:53:54,722 - INFO - joeynmt.helpers - cfg.name                           : enhi_transformer
2021-09-12 13:53:54,722 - INFO - joeynmt.helpers - cfg.data.src      

In [42]:
copy_model_to = "/content/drive/MyDrive/DA/model_" + tag + "_30epochs"
!mkdir -p $copy_model_to
os.environ["copy_model_to"] = copy_model_to

!cp -R /content/joeynmt/models/enhi_transformer_pmi_nhs_da_retrain $copy_model_to

In [43]:
# Output our validation accuracy
! cat "/content/joeynmt/models/enhi_transformer_pmi_nhs_da_retrain/validations.txt"

Steps: 69100	Loss: 50461.06250	PPL: 14.39577	bleu: 11.65630	LR: 0.00030000	
Steps: 69200	Loss: 47994.15234	PPL: 12.63606	bleu: 12.48384	LR: 0.00030000	
Steps: 69300	Loss: 46519.55469	PPL: 11.68867	bleu: 12.80169	LR: 0.00030000	
Steps: 69400	Loss: 45223.59375	PPL: 10.91488	bleu: 13.55593	LR: 0.00030000	
Steps: 69500	Loss: 44451.89844	PPL: 10.47867	bleu: 14.09237	LR: 0.00030000	
Steps: 69600	Loss: 43886.80078	PPL: 10.17034	bleu: 13.92625	LR: 0.00021000	
Steps: 69700	Loss: 43597.37500	PPL: 10.01595	bleu: 14.34961	LR: 0.00021000	
Steps: 69800	Loss: 43102.00781	PPL: 9.75713	bleu: 15.07538	LR: 0.00021000	
Steps: 69900	Loss: 42910.08594	PPL: 9.65866	bleu: 15.08302	LR: 0.00021000	
Steps: 70000	Loss: 42237.01562	PPL: 9.32112	bleu: 15.16817	LR: 0.00021000	
Steps: 70100	Loss: 42008.91406	PPL: 9.20942	bleu: 15.27796	LR: 0.00021000	
Steps: 70200	Loss: 41950.47266	PPL: 9.18102	bleu: 14.93987	LR: 0.00014700	
Steps: 70300	Loss: 41395.09375	PPL: 8.91545	bleu: 15.72386	LR: 0.00014700	
Steps: 70400	Loss:

In [44]:
# Test our model
! cd joeynmt; python3 -m joeynmt test /content/joeynmt/configs/transformer_enhi.yaml

2021-09-12 16:39:23,655 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-09-12 16:39:23,656 - INFO - joeynmt.data - Building vocabulary...
2021-09-12 16:39:24,072 - INFO - joeynmt.data - Loading dev data...
2021-09-12 16:39:24,078 - INFO - joeynmt.data - Loading test data...
2021-09-12 16:39:24,084 - INFO - joeynmt.data - Data loaded.
2021-09-12 16:39:24,113 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 300
2021-09-12 16:39:24,113 - INFO - joeynmt.prediction - Loading model from models/enhi_transformer_pmi_nhs_da_retrain/83300.ckpt
2021-09-12 16:39:28,008 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-09-12 16:39:28,203 - INFO - joeynmt.model - Enc-dec model built.
2021-09-12 16:39:28,268 - INFO - joeynmt.prediction - Decoding on dev set (data/enhi/dev.bpe.hi)...
2021-09-12 16:40:20,674 - INFO - joeynmt.prediction -  dev bleu[13a]:  18.84 [Beam search decoding with beam size = 5 and alpha = 1.0]
2021-09-12 16:40:20,6

In [45]:
!pip install sacremoses

## clone indic nlp resources repo
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
## install indic nlp library
!pip install indic-nlp-library
## export indic nlp resource library
os.environ["INDIC_RESOURCES_PATH"] = "/content/indic_nlp_resources"

fatal: destination path 'indic_nlp_resources' already exists and is not an empty directory.


In [46]:
def translate_file(input_file):
  from indicnlp.tokenize import indic_detokenize 

  config = "/content/joeynmt/models/enhi_transformer_pmi_nhs_da_retrain/config.yaml"

  !sacremoses tokenize < "$input_file" | subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$src_lang > joeynmt/translate.txt
  !cd joeynmt; python3 -m joeynmt translate "$config" < translate.txt > joey_pred.txt
  print()

  with open(input_file, 'r') as inputtext, open('joeynmt/joey_pred.txt', 'r') as trtext:
    input_lines = inputtext.readlines()
    translated_lines = trtext.readlines()

    for line1,line2 in zip(input_lines, translated_lines):
      print("Source Line: ", line1)
      print("Translated Line: ", indic_detokenize.trivial_detokenize(line2,lang='hi'))
      print()


In [47]:
input_data = """It can cause a new continuous cough, fever or loss of, or change in, sense of smell or taste (anosmia).
Generally, corona virus can cause more severe symptoms in people with weakened immune systems, older people and those with long term conditions like diabetes, cancer and chronic lung disease.
This is a rapidly changing situation which is being monitored carefully.
What you can and cannot do depends on your protection level (tier).
Glasgow and Moray are at level 3.
Most of mainland Scotland and some islands are at level 2.
Some islands are at level 1.
Check the protection level of your area
Read the Scottish Governments guidance on protection levels
Get a coronavirus test
Some people are at higher risk of developing severe illness with corona virus.
These people should strictly follow physical distancing measures.
Their household and other contacts should also strictly follow physical distancing advice.
"""

with open('input.txt', 'w') as wf:
  wf.write(input_data)

In [48]:
translate_file('input.txt')

2021-09-12 16:45:05,502 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-09-12 16:45:05,957 - INFO - joeynmt.prediction - Loading model from models/enhi_transformer_pmi_nhs_da_retrain/83300.ckpt
2021-09-12 16:45:09,857 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-09-12 16:45:10,083 - INFO - joeynmt.model - Enc-dec model built.

Source Line:  It can cause a new continuous cough, fever or loss of, or change in, sense of smell or taste (anosmia).

Translated Line:  यह एक नयी निरंतर खाँसी, ज्वर या कमी उत्पन्न कर सकता है, या मस्तिष्क या तनाव की संवेदनशीलता में परिवर्तन कर सकता है।


Source Line:  Generally, corona virus can cause more severe symptoms in people with weakened immune systems, older people and those with long term conditions like diabetes, cancer and chronic lung disease.

Translated Line:  सामान्यतः, कोरोनावायरस कमजोर प्रतिरक्षी तंत्र वाले लोगों में अधिक गंभीर लक्षण उत्पन्न कर सकता है और वे मधुमेह, कैंसर और चिरकालिक रोग जैसी दीर्घकालिक स्थितियो

In [ ]:
!python3 /content/joeynmt/scripts/plot_validations.py /content/joeynmt/models/enhi_transformer_pmi_nhs_da --plot_values bleu PPL --output_path bleu-ppl.png